Scenario

You are working as an analyst with this internet service provider. You are provided with this historical data about your company's customers and their churn trends. Your task is to build a machine learning model that will help the company identify customers that are more likely to default/churn and thus prevent losses from such customers.

Instructions

In this lab, we will first take a look at the degree of imbalance in the data and correct it using the techniques we learned on the class.

1. Import the required libraries and modules that you would need.

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

2. Read that data into Python and call the dataframe churnData.

In [115]:
churnData = pd.read_csv('files_for_lab/Customer-Churn.csv')

In [72]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


3. Check the datatypes of all the columns in the data. You would see that the column TotalCharges is object type. Convert this column into numeric type using pd.to_numeric function.

In [73]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [116]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

4. Check for null values in the dataframe. Replace the null values.

In [107]:
churnData.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [108]:
churnData.TotalCharges.value_counts(dropna=False)

NaN        11
20.20      11
19.75       9
20.05       8
19.90       8
           ..
6849.40     1
692.35      1
130.15      1
3211.90     1
6844.50     1
Name: TotalCharges, Length: 6531, dtype: int64

In [96]:
churnData.TotalCharges.dtype

dtype('float64')

In [118]:
churnData = churnData.fillna(np.mean(churnData['TotalCharges']))

In [119]:
churnData.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

5. Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

In [164]:
X = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']].copy()

In [121]:
X.head()

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,1,0,29.85,29.85
1,34,0,56.95,1889.50
2,2,0,53.85,108.15
3,45,0,42.30,1840.75
4,2,0,70.70,151.65


a. Scale the features either by using normalizer or a standard scaler.

In [123]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler().fit(X)
X_scaled = ss.transform(X)

b. Split the data into a training set and a test set.

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, churnData['Churn'], random_state=0)

c. Fit a logistic regression model on the training data.

In [128]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression().fit(X_train, y_train)
clf.score(X_train, y_train)

0.7913669064748201

d. Check the accuracy on the test data.

In [137]:
clf.score(X_test, y_test)

0.7830777967064169

Managing imbalance in the dataset

6. Check for the imbalance.

In [138]:
y.value_counts()

No     5174
Yes    1869
Name: Churn, dtype: int64

7. Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.

In [150]:
X_scaled = pd.DataFrame(X_scaled)
data = pd.concat([X_scaled,y], axis=1)

In [158]:
majority_class = data[data['Churn'] == 'No'] 
minority_class = data[data['Churn'] == 'Yes']
from sklearn.utils import resample

In [159]:
#Upsampling
minority_class_oversampled = resample(minority_class, 
                                  replace=True, 
                                  n_samples = len(majority_class))

churnData_upsampled = pd.concat([majority_class, minority_class_oversampled], axis=0)

In [160]:
#Downsampling
majority_class_undersampled = resample(majority_class, 
                                  replace=True, 
                                  n_samples = len(minority_class))

churnData_downsampled = pd.concat([minority_class, majority_class_undersampled], axis=0)

8. Each time fit the model and see how the accuracy of the model is.

In [161]:
# upsampled model
y = churnData_upsampled['Churn']
X = churnData_upsampled.drop(['Churn'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf_u = LogisticRegression().fit(X_train, y_train)
print('train data accuracy: ', clf_u.score(X_train, y_train))
print('test data accuracy: ', clf_u.score(X_test, y_test))


train data accuracy:  0.7295451617059657
test data accuracy:  0.7363741785852339


In [162]:
# downsampled model
y = churnData_downsampled['Churn']
X = churnData_downsampled.drop(['Churn'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

clf_d = LogisticRegression().fit(X_train, y_train)
print('train data accuracy: ', clf_d.score(X_train, y_train))
print('test data accuracy: ', clf_d.score(X_test, y_test))


train data accuracy:  0.7377809489832322
test data accuracy:  0.7315508021390374
